In [287]:
import tensorflow as tf
import keras
import numpy as np
import os
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['axes.labelsize'] = 14
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12

import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
import seaborn as sns

import scipy.stats as sc_stats

import random

from sklearn.model_selection import train_test_split
onehot_encoder=OneHotEncoder(sparse=False)

In [288]:
# parameters
time_steps = 1
inputs = 8
outputs = 2

In [289]:
def add_releveant_features(task_df):


    task_df['PrevOutcome']=task_df['Outcome'].shift(1)
    task_df.loc[1,'PrevOutcome']= 0

    task_df['PrevChoice']=task_df['Choice'].shift(1)
    task_df.loc[1,'PrevChoice']= 0

    task_df['PrevSafe']=task_df['Safe'].shift(1)
    task_df.loc[1,'PrevSafe']= 0

    task_df['PrevBigRisky']=task_df['BigRisky'].shift(1)
    task_df.loc[1,'PrevBigRisky']= 0

    task_df['PrevSmallRisky']=task_df['SmallRisky'].shift(1)
    task_df.loc[1,'PrevSmallRisky']= 0
    
#     task_df['PrevRT']=task_df['RT'].shift(1)
#     task_df.loc[1,'PrevRT']= N
    
    
    
    return task_df

In [318]:
stop=150
num_batches=1000
seq_len=3

def data_split(task_df,dopa_task_df):
#     stop = 200

    stop = 150

#     stop=300

#     stop = 750

#     stop=1500


    print(task_df.shape)
 

    ##----------------- UNCOMMENT BELOW
    
    
#     train_X = task_df.loc[task_df.TrialNum!=0,['Safe','BigRisky','SmallRisky']].values
#     train_y = task_df.loc[task_df.TrialNum!=0,['Choice']].values.astype(np.int32)
    
#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum!=0,['Safe','BigRisky','SmallRisky']].values
#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum!=0,['Choice']].values.astype(np.int32)

    
    
#     train_X = task_df.loc[task_df.TrialNum!=0,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice']].values

#     train_y = task_df.loc[task_df.TrialNum!=0,['Choice']].values.astype(np.int32)


#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum!=0,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice']].values

#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum!=0,['Choice']].values.astype(np.int32)



#     train_X = task_df.loc[task_df.TrialNum>1, ['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice']].values
#     train_y = task_df.loc[task_df.TrialNum>1,['Choice']].values.astype(np.int32)

#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice']].values
#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Choice']].values.astype(np.int32)


####### Prev O + C+ R + CurrO--------------------
    
#     train_X = task_df.loc[task_df.TrialNum!=0,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
#     train_y = task_df.loc[task_df.TrialNum!=0,['Choice']].values.astype(np.int32)

#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum!=0,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum!=0,['Choice']].values.astype(np.int32)

    train_X = task_df.loc[task_df.TrialNum>1, ['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
    train_y = task_df.loc[task_df.TrialNum>1,['Choice']].values.astype(np.int32)

    test_X = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
    test_y = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Choice']].values.astype(np.int32)

    #### PLACEHOLDER VAL_X
    val_X = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky']].values
    val_y = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Choice']].values.astype(np.int32)


#### - Prev RT+C+R+O + Curr O----------------------

#     train_X = task_df.loc[task_df.TrialNum>1, ['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     train_y = task_df.loc[task_df.TrialNum>1,['Choice']].values.astype(np.int32)

#     test_X = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Safe','BigRisky','SmallRisky','PrevOutcome','PrevChoice','PrevSafe','PrevBigRisky','PrevSmallRisky','PrevRT']].values
#     test_y = dopa_task_df.loc[dopa_task_df.TrialNum>1,['Choice']].values.astype(np.int32)



    ###### when splitting data into train and validation

    # train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.35, random_state=1)

    
#     train_X, val_X, train_y, val_y = train_X[:stop], train_X[stop:], train_y[:stop], train_y[stop:]


####################
 ##### splitting data into train test valid from the same dataset ###############
    
#     train_X, val_X, test_X, train_y, val_y, test_y = train_X[:stop], train_X[stop:stop+int(stop/2)], train_X[stop+int(stop/2):], train_y[:stop], train_y[stop:stop+int(stop/2)], train_y[stop+int(stop/2):],

#### switching the order for test and val
#     half = 1

#     if half==1:
#         train_X, test_X, val_X, train_y, test_y, val_y= train_X[:stop], train_X[stop:stop+int(stop/2)], train_X[stop+int(stop/2):], train_y[:stop], train_y[stop:stop+int(stop/2)], train_y[stop+int(stop/2):],
    
#     else:
    
#         train_X, test_X, val_X, train_y, test_y, val_y= train_X[stop-1:], train_X[:int(stop/2)], train_X[int(stop/2):stop-1], train_y[stop-1:], train_y[:int(stop/2)], train_y[int(stop/2):stop-1]

#     ##############

    
    
#     print(train_X)
    
    
    ## Combining PLAC + LDOPA datasets
#     train_X, test_X, val_X, train_y, test_y, val_y = np.concatenate(train_X[:stop],test_X[:stop]), np.concatenate(train_X[stop:stop+int(stop/2)])#,test_X[stop:stop+int(stop/2)]),np.concatenate(train_X[stop+int(stop/2):],test_X[stop+int(stop/2):]),np.concatenate(train_y[:stop],test_y[:stop]), np.concatenate(train_y[stop:stop+int(stop/2)],test_y[stop:stop+int(stop/2)]), np.concatenate(train_y[stop+int(stop/2):],test_y[stop+int(stop/2):])
    

    
    
#     train_X, test_X,val_X = np.concatenate((train_X[:stop],test_X[:stop]),axis=0), np.concatenate((train_X[stop:stop+int(stop/2)] ,test_X[stop:stop+int(stop/2)]),axis=0), np.concatenate((train_X[stop+int(stop/2):],test_X[stop+int(stop/2):]),axis=0)
#     train_y, test_y,val_y = np.concatenate((train_y[:stop],test_y[:stop]),axis=0), np.concatenate((train_y[stop:stop+int(stop/2)] ,test_y[stop:stop+int(stop/2)]),axis=0), np.concatenate((train_y[stop+int(stop/2):],test_y[stop+int(stop/2):]),axis=0)



#     ## blocking data 
#     train_X_aside, train_y_aside, test_X_aside, test_y_aside  = train_X, train_y, test_X, test_y 
    
#     train_X= np.concatenate((build_dataset_train(train_X_aside),build_dataset_train(test_X_aside)), axis=0)
#     train_y= np.concatenate((build_dataset_train(train_y_aside),build_dataset_train(test_y_aside)), axis=0)

    
#     val_X= np.concatenate((build_dataset_valid(train_X_aside),build_dataset_valid(test_X_aside)), axis=0)
#     val_y= np.concatenate((build_dataset_valid(train_y_aside),build_dataset_valid(test_y_aside)), axis=0)

    
#     test_X= np.concatenate((build_dataset_test(train_X_aside),build_dataset_test(test_X_aside)), axis=0)
#     test_y= np.concatenate((build_dataset_test(train_y_aside),build_dataset_test(test_y_aside)), axis=0)
    

        ### CHUNK SPLITTING

        
        
    train_X_aside, train_y_aside, test_X_aside, test_y_aside  = train_X, train_y, test_X, test_y 
    
    train_X= np.concatenate((chunk_split_data(train_X_aside,0,10),chunk_split_data(test_X_aside,0,10)), axis=0)
    train_y= np.concatenate((chunk_split_data(train_y_aside,0,10),chunk_split_data(test_y_aside,0,10)), axis=0)

    
    val_X= np.concatenate((chunk_split_data(train_X_aside,15,20),chunk_split_data(test_X_aside,15,20)), axis=0)
    val_y= np.concatenate((chunk_split_data(train_y_aside,15,20),chunk_split_data(test_y_aside,15,20)), axis=0)

    test_X= np.concatenate((chunk_split_data(train_X_aside,10,15),chunk_split_data(test_X_aside,10,15)), axis=0)
    test_y= np.concatenate((chunk_split_data(train_y_aside,10,15),chunk_split_data(test_y_aside,10,15)), axis=0)

    
    
    
#     ##### FURTHER TRAINING WITH SUBSEQUENCES WITH REPLACEMENT
#     X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)
#     for k in range(num_batches):
#             X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)
#             train_X = np.concatenate((train_X,X_seq), axis=0)
# #             print(train_X.shape)
#             train_y = np.concatenate((train_y, y_seq),axis=0)


#     X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)
#     train_X, train_y = X_seq, y_seq
#     for k in range(num_batches-1):
#             X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)
#             train_X = np.concatenate((train_X,X_seq), axis=0)
# #             print(train_X.shape)
#             train_y = np.concatenate((train_y, y_seq),axis=0)
# # # # # ##########################







    #### PRE TRAINING
#     stop = int(0.7*len(train_X))
#     print(stop)
#     train_X, test_X, val_X, train_y, test_y, val_y= train_X[:stop], train_X[stop:stop+int((len(train_X)-stop)/2)], train_X[stop+int((len(train_X)-stop)/2):],train_y[:stop], train_y[stop:stop+int((len(train_X)-stop)/2)], train_y[stop+int((len(train_X)-stop)/2):]
    
#     train_X, test_X, val_X, train_y, test_y, val_y = train_X, test_X, test_X, train_y, test_y, test_y
    ###################################################################


    print(train_X.shape)
    print(train_y.shape)
    print(val_X.shape)
    print(val_y.shape)
    print(test_X.shape)
    print(test_y.shape)

    # # center and scale
    scaler = MinMaxScaler(feature_range=(0, 1))    
    train_X = scaler.fit_transform(train_X)
    test_X = scaler.fit_transform(test_X)
    val_X = scaler.fit_transform(val_X)


    train_X = train_X[:,None,:]
    val_X = val_X[:,None,:]
    test_X = test_X[:,None,:]


    # # one-hot encode the outputs

    onehot_encoder = OneHotEncoder()
    encode_categorical = train_y.reshape(len(train_y), 1)
    encode_categorical_test = test_y.reshape(len(test_y), 1)
    encode_categorical_val = val_y.reshape(len(val_y),1)


    train_y = onehot_encoder.fit_transform(encode_categorical).toarray()
    test_y = onehot_encoder.fit_transform(encode_categorical_test).toarray()
    val_y = onehot_encoder.fit_transform(encode_categorical_val).toarray()

    
    return train_X, train_y, test_X, test_y, val_X,val_y
#     return train_X, test_X, val_X#, test_X, test_y, val_X,val_y

def build_dataset_train(data):
    
    return np.concatenate((data[:int(stop/3)],data[2*int(stop/3):int(stop)], data[stop+int(stop/3):stop+2*int(stop/3)]), axis=0)
    

def build_dataset_test(data):
    
    return np.concatenate((data[int(stop/3):int(stop/3)+int(stop/6)], data[int(stop):int(stop)+int(stop/6)], data[stop+2*int(stop/3): stop+2*int(stop/3) + int(stop/6) ]), axis=0)

def build_dataset_valid(data):
    
    return np.concatenate((data[int(stop/3)+int(stop/6):int(stop/3)+2*int(stop/6)], data[int(stop)+int(stop/6):int(stop)+2*int(stop/6)], data[stop+2*int(stop/3) + int(stop/6) : ]), axis=0)

In [319]:
def chunk_split_data(data,start_chunk,end_chunk):
    
    a=[k for k in range(start_chunk,end_chunk)]
    out=[]

    for d in range(0,data.shape[0],20):

        c= [c+d for c in a]
        out = out+c

    while out[-1]>=data.shape[0]-1:
        out.pop()
#     return out
    return data[out]

In [320]:
# chunk_split_data(train_X,0,10)
# chunk_split_data(train_X,10,15)
# chunk_split_data(train_X,15,20)


In [321]:
def train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y): 
    reset_graph()

    learning_rate = 0.001
    epochs = 50000
    batch_size = int(train_X.shape[0]/2)
    # batch_size = 100
    length = train_X.shape[0]
    display = 100
    neurons = neurons

    num_batches = 100
    seq_len = 10

    percent_above_PT = 1

    train_threshold = PT_R2 + percent_above_PT


    save_step = 100


    best_loss_val = np.infty
    checks_since_last_progress = 0
    max_checks_without_progress = 1000


    # clear graph (if any) before running
    tf.reset_default_graph()

    X = tf.placeholder(tf.float32, [None, time_steps, inputs])

    y = tf.placeholder(tf.float32, [None, outputs])

    # LSTM Cell
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=neurons, activation=tf.nn.relu)
    cell_outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

    # pass into Dense layer
    stacked_outputs = tf.reshape(cell_outputs, [-1, neurons])
    out = tf.layers.dense(inputs=stacked_outputs, units=outputs)

    probability = tf.nn.softmax(out)

    # squared error loss or cost function for linear regression
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
            labels=y, logits=out))

    # optimizer to minimize cost
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
    training_op = optimizer.minimize(loss)
    saver = tf.train.Saver()
    
    accuracy = tf.metrics.accuracy(labels =  tf.argmax(y, 1),
                          predictions = tf.argmax(out, 1),
                          name = "accuracy")
    precision = tf.metrics.precision(labels=tf.argmax(y, 1),
                                 predictions=tf.argmax(out, 1),
                                 name="precision")
    recall = tf.metrics.recall(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="recall")
    f1 = 2 * accuracy[1] * recall[1] / ( precision[1] + recall[1] )

    acc_up,acc_val = accuracy
    auc = tf.metrics.auc(labels=tf.argmax(y, 1),
                           predictions=tf.argmax(out, 1),
                           name="auc")
    
    valid_store = []
    
    with tf.Session() as sess:
        #######################
#         saver.restore(sess, "./checkpts/Original_RNN_LSTM_8features_v2.ckpt")
#         saver.restore(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")


        saver.restore(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")

        #######################
        
        # initialize all variables
        tf.global_variables_initializer().run()
        tf.local_variables_initializer().run()

        # Train the model
        for steps in range(epochs):
            mini_batch = zip(range(0, length, batch_size),
                       range(batch_size, length+1, batch_size))

            # train data in mini-batches
            for (start, end) in mini_batch:
    #             print(start,end)
                sess.run(training_op, feed_dict = {X: train_X[start:end,:,:],
                                                   y: train_y[start:end,:]}) 

            ## train data in batches of length subsequence

    #         for k in range(num_batches):
    #             X_seq, y_seq = random_subsequence(train_X,train_y,seq_len)

    #             sess.run(training_op, feed_dict = {X:X_seq,y:y_seq}) 
            loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
            loss_val = loss.eval(feed_dict={X: val_X, y: val_y})


            # print training performance 
            if (steps+1) % display == 0:
                # evaluate loss function on training set


                loss_fn = loss.eval(feed_dict = {X: train_X, y: train_y})
                print('Step: {}  \tTraining loss: {}'.format((steps+1), loss_fn))

                acc_train = acc_val.eval(feed_dict={X: train_X, y: train_y})
                print('Step: {}  \tTraining accuracy: {}'.format((steps+1), acc_train))


                acc_test = acc_val.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest accuracy: {}'.format((steps+1), acc_test))

                loss_test = loss.eval(feed_dict={X: test_X, y: test_y})
    #             print('Step: {}  \tTest loss: {}'.format((steps+1), loss_test))

                accu_val = acc_val.eval(feed_dict={X: val_X, y: val_y})

                loss_val = loss.eval(feed_dict={X: val_X, y: val_y})
                print('Step: {}  \tValid loss: {}'.format((steps+1), loss_val))

                valid_store.append(loss_val)







            if (1 + loss_fn/np.log(0.5)) > train_threshold:
                    print("Threshold achieved, quit training")
                    break


            if loss_val < best_loss_val:

                        best_loss_val = loss_val
                        checks_since_last_progress = 0
            else:
                            checks_since_last_progress += 1


            # EARLY STOPPING
            if checks_since_last_progress > max_checks_without_progress:
                print("Early stopping!")
                break


            if (steps+1) % save_step ==0:
                                save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")

#                 save_path = saver.save(sess, "./checkpts/RNN_Internet_LSTM_model_5features.ckpt")





    #     evaluate model accuracy
        acc, prec, recall, f1, AUC = sess.run([accuracy, precision, recall, f1,auc],
                                         feed_dict = {X: train_X, y: train_y})
        prob_train = probability.eval(feed_dict = {X: train_X, y: train_y})
        prob_test = probability.eval(feed_dict = {X: test_X, y: test_y})
        prob_valid = probability.eval(feed_dict = {X: val_X, y: val_y})



        print('\nEvaluation  on training set')
        print('Accuracy:', acc[1])
        print('Precision:', prec[1])
        print('Recall:', recall[1])
        print('F1 score:', f1)
        print('AUC:', AUC[1])
        
        
#         save_path = saver.save(sess, "./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt")
        save_path = saver.save(sess, "./checkpts/Later_v2_DATA_RNN_LSTM_8features.ckpt")

        
#         save_path = saver.save(sess, "./checkpts/OriginalDATA_RNN_LSTM_8features.ckpt")
#         save_path = saver.save(sess, "./checkpts/LaterDATA_RNN_LSTM_8features.ckpt")


#         save_path = saver.save(sess, "./checkpts/Original_RNN_LSTM_8features.ckpt")

#         save_path = saver.save(sess, "./checkpts/Later_RNN_LSTM_8features.ckpt")


    metric_out_df= pd.DataFrame(np.array([acc[1],prec[1],recall[1],f1,AUC[1],loss_fn,accu_val,best_loss_val,acc_test,loss_test,neurons,learning_rate,epochs,steps]).reshape(-1,14),columns =["accuracy","precision","recall","f1_score","auc","loss","accuracy_val","loss_val","accuracy_test","loss_test","neurons","learning_rate","n_epochs","steps"])
    return metric_out_df, prob_train, prob_test, prob_valid
    


    
    
    
    
def random_subsequence(X,y,seq_len):
    rnd  = random.randint(0,len(X)-seq_len)
    X_seq, y_seq = X[rnd:rnd+seq_len,:], y[rnd:rnd+seq_len,:]
    return X_seq, y_seq

    print(y_seq.shape)

In [322]:
# file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/"
subj_num =11
file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/generated_data_subj29_params.csv"

file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/mu=1/generateddata300mu1params.csv"

# file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/mu=0.5/generateddata300mu0_5params.csv"

### ACTUAL DATA
file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num) + "/experiment_data.csv"
file_dopa_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/dopadata/subject_num_"+str(subj_num) +"/dopa_experiment_data.csv"
    
task_df = pd.read_csv(file_name)
dopa_task_df = pd.read_csv(file_dopa_name)

task_df = add_releveant_features(task_df)
dopa_task_df = add_releveant_features(dopa_task_df)
    


task_df.head(10)


# train_data = np.concatenate((build_dataset_train(task_df.loc[task_df.TrialNum>1]),build_dataset_train(dopa_task_df.loc[dopa_task_df.TrialNum>1])),axis=0)
# train_data_df = pd.DataFrame(train_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])

# test_data = np.concatenate((build_dataset_test(task_df.loc[task_df.TrialNum>1]),build_dataset_test(dopa_task_df.loc[dopa_task_df.TrialNum>1])),axis=0)
# test_data_df = pd.DataFrame(test_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])



train_X, train_y, test_X, test_y,val_X,val_y = data_split(task_df,dopa_task_df)


# train_X.shape

(301, 15)
(300, 8)
(300, 1)
(146, 8)
(146, 1)
(150, 8)
(150, 1)


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

In [323]:
os.mkdir("/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/v10chunks/")

In [424]:
for subj_num in range(28,42):
    print("Subject"+ str(subj_num))

#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/v2/subject_num_"+str(subj_num)
#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/vcheck/subject_num_"+str(subj_num)

#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/v6chunks/subject_num_"+str(subj_num)

    file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/v10chunks/subject_num_"+str(subj_num)

#     os.mkdir(file_path)

    
    file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num) + "/experiment_data.csv"
    file_dopa_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/dopadata/subject_num_"+str(subj_num) +"/dopa_experiment_data.csv"
    
    task_df = pd.read_csv(file_name)
    dopa_task_df = pd.read_csv(file_dopa_name)

    task_df = add_releveant_features(task_df)
    dopa_task_df = add_releveant_features(dopa_task_df)
    
    
    ## blocking ####
#     train_data = np.concatenate((build_dataset_train(task_df.loc[task_df.TrialNum>1]),build_dataset_train(dopa_task_df.loc[dopa_task_df.TrialNum>1])),axis=0)
#     train_data_df = pd.DataFrame(train_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])

#     test_data = np.concatenate((build_dataset_test(task_df.loc[task_df.TrialNum>1]),build_dataset_test(dopa_task_df.loc[dopa_task_df.TrialNum>1])),axis=0)
#     test_data_df = pd.DataFrame(test_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])
    
#     val_data = np.concatenate((build_dataset_valid(task_df.loc[task_df.TrialNum>1]),build_dataset_valid(dopa_task_df.loc[dopa_task_df.TrialNum>1])),axis=0)
#     val_data_df = pd.DataFrame(val_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])
#     ##########

    ## CHUNKING
    train_X_aside = task_df.loc[task_df.TrialNum>1].values; test_X_aside= dopa_task_df.loc[task_df.TrialNum>1].values;
    
    
    train_data = np.concatenate((chunk_split_data(train_X_aside,0,10),chunk_split_data(test_X_aside,0,10)), axis=0)
    train_data_df = pd.DataFrame(train_data, columns=task_df.columns)

    val_data= np.concatenate((chunk_split_data(train_X_aside,15,20),chunk_split_data(test_X_aside,15,20)), axis=0)
    val_data_df = pd.DataFrame(val_data, columns=task_df.columns)

    test_data= np.concatenate((chunk_split_data(train_X_aside,10,15),chunk_split_data(test_X_aside,10,15)), axis=0)
    test_data_df = pd.DataFrame(test_data, columns = task_df.columns)


# train_data= np.concatenate((chunk_split_data(train_X_aside,0,10),chunk_split_data(test_X_aside,0,10)), axis=0)
#     train_data_df = pd.DataFrame(train_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])

    
#     val_data= np.concatenate((chunk_split_data(train_X_aside,15,20),chunk_split_data(test_X_aside,15,20)), axis=0)
#     val_data_df = pd.DataFrame(val_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])

#     test_data= np.concatenate((chunk_split_data(train_X_aside,10,15),chunk_split_data(test_X_aside,10,15)), axis=0)
#     test_data_df = pd.DataFrame(test_data[:,:10],columns =['TrialNum','SideOfScreen','Safe','BigRisky','SmallRisky','SideChosen','Choice','Outcome','RT','Happiness'])

#     print(train_data.shape)
#     print(val_data.shape)
#     print(test_data.shape)
    
    
    train_data_df.to_csv(file_path+"/train_data.csv")
    test_data_df.to_csv(file_path+"/test_data.csv")
    val_data_df.to_csv(file_path+"/val_data.csv")


Subject28
Subject29
Subject30
Subject31
Subject32
Subject33
Subject34
Subject35
Subject36
Subject37
Subject38
Subject39
Subject40
Subject41


In [421]:
train_data_df

,TrialNum,SideOfScreen,Safe,BigRisky,SmallRisky,SideChosen,Choice,Outcome,RT,Happiness,PrevOutcome,PrevChoice,PrevSafe,PrevBigRisky,PrevSmallRisky
0,2.0,1.0,0.0,45.0,-30.0,1.0,1.0,45.0,1.727,NaN,-35.0,0.0,-35.0,0.0,-111.0
1,3.0,1.0,-15.0,0.0,-47.0,2.0,0.0,-15.0,1.198,0.535156,45.0,1.0,0.0,45.0,-30.0
2,4.0,1.0,30.0,108.0,0.0,1.0,1.0,108.0,1.202,NaN,-15.0,0.0,-15.0,0.0,-47.0
3,5.0,1.0,-35.0,0.0,-70.0,2.0,0.0,-35.0,4.878,NaN,108.0,1.0,30.0,108.0,0.0
4,6.0,1.0,0.0,80.0,-32.0,1.0,1.0,80.0,2.249,0.541667,-35.0,0.0,-35.0,0.0,-70.0
5,7.0,1.0,0.0,30.0,-25.0,1.0,1.0,-25.0,3.007,NaN,80.0,1.0,0.0,80.0,-32.0
6,8.0,1.0,25.0,70.0,0.0,1.0,1.0,70.0,1.223,NaN,-25.0,1.0,0.0,30.0,-25.0
7,9.0,1.0,35.0,126.0,0.0,1.0,1.0,0.0,0.902,0.518229,70.0,1.0,25.0,70.0,0.0
8,10.0,1.0,30.0,60.0,0.0,2.0,0.0,30.0,3.523,NaN,0.0,1.0,35.0,126.0,0.0
9,11.0,2.0,0.0,30.0,-20.0,1.0,0.0,0.0,3.399,NaN,30.0,0.0,30.0,60.0,0.0


In [156]:
# file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/vcheck/"
# os.mkdir(file_path)

In [414]:
neurons = 8
for subj_num in range(36,37):
# for subj_num in range(28,42):

# for subj_num in [20]:##[15, 16, 17]:# 19, 20, 23, 24, 25, 26, 29, 36, 37, 40]:

    print("Subject"+ str(subj_num))

    file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num)+"/"
    file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num) + "/experiment_data.csv"
    file_dopa_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/dopadata/subject_num_"+str(subj_num) +"/dopa_experiment_data.csv"
    
    task_df = pd.read_csv(file_name)
    dopa_task_df = pd.read_csv(file_dopa_name)

    task_df = add_releveant_features(task_df)
    dopa_task_df = add_releveant_features(dopa_task_df)
    
    PT_file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/"
    PT_file_name = PT_file_path  + "PT_loss_updated.csv"
    PT_metrics = pd.read_csv(PT_file_name)
    PT_metrics = PT_metrics[PT_metrics.PT_loss !=0]
    PT_R2= PT_metrics[PT_metrics.Subject_number ==subj_num].PT_pseudoR2.iloc[0]
    
    
    
    # for generated data ##
#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/"
#     file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/generated_data_subj29_params.csv"
#     file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/generated_data_1500_subj29_params.csv"

#     file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/generated_data_3000_subj29_params.csv"


#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/mu=1"
#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/mu=0.5"


#     file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/mu=1/generateddata1500mu1params.csv"
#     file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/mu=0.5/generateddata300mu0_5params.csv"



############# ORIGINAL ##############
#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/FittingProc/mu=0.5"

#     file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/FittingProc/mu=0.5/generateddata3000mu0_5params.csv"


# ############ ACTUAL ###############

#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/FittingProc/mu=0.5"

#     file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/generated_data/PT_fake_data/FittingProc/mu=0.5/generateddata600mu0_5params.csv"


#     task_df=pd.read_csv(file_name)
# #     task_df.TrialNum = task_df.TrialNum-1

#     task_df = add_releveant_features(task_df)

    
    
    
    
    
    ### ORIGINAL
#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining"

#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining_v2"


#     task_df = task_net_df; dopa_task_df = task_net_df;
    
    #############################
#     task_df = task_net_train_df
#     dopa_task_df  = task_net_valid_df

    
    
     
    ### ACTUAL DATA PER SUBJECT
#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/vcheck/subject_num_"+str(subj_num)

#     file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/subject_num_"+str(subj_num)+"/first_half_train"

    
    file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/v10chunks/subject_num_"+str(subj_num)

    
    train_X, train_y, test_X, test_y,val_X,val_y = data_split(task_df,dopa_task_df)

    
    
#     train_X,train_y= data_split(task_df,dopa_task_df)

    
#     X_seq, y_seq = random_subsequence(train_X,train_y,10)

    metric_out_df, prob_train, prob_test, prob_val = train_RNN(neurons,train_X,train_y,test_X,test_y,val_X,val_y)
    
    print(metric_out_df)
    
    
    
#     os.mkdir(file_path + "50_splits_combined_1sthalf")
#     os.mkdir(file_path + "50_splits_combined_2ndhalf")


    
#     os.mkdir(file_path + "combined_1sthalf")
#     os.mkdir(file_path + "combined_2ndhalf")




#     os.mkdir(file_path + "300_sub29")
#     os.mkdir(file_path + "1500_sub29")
#     os.mkdir(file_path + "3000_sub29")

#     os.mkdir(file_path + "/300")
#     os.mkdir(file_path + "/1500")
#     os.mkdir(file_path + "/3000")
#     os.mkdir(file_path + "/600")



#     os.mkdir(file_path)



    
#     metric_out_df.to_csv(file_path+"LSTM_updated_Crossval_curr_metrics_neurons="+str(neurons)+".csv")

#     metric_out_df.to_csv(file_path+"LSTM_updated_Crossval_currprev_v2_metrics_neurons="+str(neurons)+".csv")

########################################
#     metric_out_df.to_csv(file_path+"LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")
#######################################
    
#     metric_out_df.to_csv(file_path+"LSTM_updated_Crossval_currprev_RT_opts_metrics_neurons="+str(neurons)+".csv")

#     metric_out_df.to_csv(file_path+"LSTM_updated_CrossvalTESTinsess1sthalf_currprev_opts_metrics_neurons="+str(neurons)+".csv")
#     metric_out_df.to_csv(file_path+"LSTM_updated_CrossvalTESTinsess2ndhalf_currprev_opts_metrics_neurons="+str(neurons)+".csv")


#     metric_out_df.to_csv(file_path+"combined_1sthalf/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")
#     metric_out_df.to_csv(file_path+"combined_2ndhalf/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")



#     metric_out_df.to_csv(file_path+"50_splits_combined_1sthalf/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")
#     metric_out_df.to_csv(file_path+"50_splits_combined_2ndhalf/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")



##### generated data ######
#     metric_out_df.to_csv(file_path+"300_sub29/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")
#     metric_out_df.to_csv(file_path+"1500_sub29/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")
#     metric_out_df.to_csv(file_path+"3000_sub29/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")

#     metric_out_df.to_csv(file_path+"3000_sub29/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")

#     metric_out_df.to_csv(file_path+"/300/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")
#     metric_out_df.to_csv(file_path+"/1500/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")

#     metric_out_df.to_csv(file_path+"/3000/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+".csv")

#     metric_out_df.to_csv(file_path+"/600/LSTM_updated_Crossval_currprev_opts_metrics_neurons="+str(neurons)+".csv")


#     metric_out_df.to_csv(file_path+"/3000/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+"_v2.csv")
#     metric_out_df.to_csv(file_path+"/600/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+"_v2.csv")


#     metric_out_df.to_csv(file_path+"/600/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+".csv")


    metric_out_df.to_csv(file_path+"/LSTM_updated_Crossval_currprev_opts_metricsneurons="+str(neurons)+".csv")


    
    prob_train_df = pd.DataFrame(prob_train.reshape(-1,2),columns = {'action_0','action_1'})
    prob_test_df = pd.DataFrame(prob_test.reshape(-1,2),columns = {'action_0','action_1'})
    prob_val_df = pd.DataFrame(prob_val.reshape(-1,2),columns = {'action_0','action_1'})

#     prob_train_df.to_csv(file_path + "prob_train_currentopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "prob_test_currentopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "prob_val_currentopts_neurons="+str(neurons)+".csv")

#     prob_train_df.to_csv(file_path + "prob_train_currentopts_prev_outchoicevv2_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "prob_test_currentopts_prev_outchoicev2_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "prob_val_currentopts_prev_outchoicev2_neurons="+str(neurons)+".csv")

################################
    prob_train_df.to_csv(file_path + "/prob_train_currentprevopts_neurons="+str(neurons)+".csv")
    prob_test_df.to_csv(file_path + "/prob_test_currentprevopts_neurons="+str(neurons)+".csv")
    prob_val_df.to_csv(file_path + "/prob_val_currentprevopts_neurons="+str(neurons)+".csv")
#############################

    

#     prob_train_df.to_csv(file_path + "1sthalf/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "1sthalf/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "1sthalf/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")

    

#     prob_train_df.to_csv(file_path + "2ndhalf/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "2ndhalf/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "2ndhalf/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")

#     prob_train_df.to_csv(file_path + "combined_1sthalf/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "combined_1sthalf/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "combined_1sthalf/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")

#     prob_train_df.to_csv(file_path + "combined_2ndhalf/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "combined_2ndhalf/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "combined_2ndhalf/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")


#     prob_train_df.to_csv(file_path + "50_splits_combined_1sthalf/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "50_splits_combined_1sthalf/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "50_splits_combined_1sthalf/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")






####### Generated data #######
#     prob_train_df.to_csv(file_path + "300_sub29/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "300_sub29_combined_1sthalf/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "300_sub29/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")


#     prob_train_df.to_csv(file_path + "1500_sub29/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "1500_sub29/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "1500_sub29/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")

#     prob_train_df.to_csv(file_path + "3000_sub29/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "3000_sub29/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "3000_sub29/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")



#     prob_train_df.to_csv(file_path + "/300/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "/300/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "/300/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")

#     prob_train_df.to_csv(file_path + "/1500/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "/1500/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "/1500/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")

#     prob_train_df.to_csv(file_path + "/3000/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "/3000/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "/3000/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")



#     prob_train_df.to_csv(file_path + "/600/" +"prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df.to_csv(file_path + "/600/"+ "prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df.to_csv(file_path + "/600/"+"prob_val_currentprevopts_neurons="+str(neurons)+".csv")





Subject36
(301, 15)
(300, 8)
(300, 1)
(146, 8)
(146, 1)
(150, 8)
(150, 1)


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

INFO:tensorflow:Restoring parameters from ./checkpts/Original_v2_DATA_RNN_LSTM_8features.ckpt
Step: 100  	Training loss: 0.5768251419067383
Step: 100  	Training accuracy: 0.746666669845581
Step: 100  	Valid loss: 0.6119353175163269
Step: 200  	Training loss: 0.5098356604576111
Step: 200  	Training accuracy: 0.734375
Step: 200  	Valid loss: 0.5775622129440308
Step: 300  	Training loss: 0.47863465547561646
Step: 300  	Training accuracy: 0.7419571280479431
Step: 300  	Valid loss: 0.562315046787262
Step: 400  	Training loss: 0.4497145116329193
Step: 400  	Training accuracy: 0.7461685538291931
Step: 400  	Valid loss: 0.5404934883117676
Step: 500  	Training loss: 0.4211510717868805
Step: 500  	Training accuracy: 0.754470944404602
Step: 500  	Valid loss: 0.5153321623802185
Step: 600  	Training loss: 0.3945452868938446
Step: 600  	Training accuracy: 0.7621951103210449
Step: 600  	Valid loss: 0.4909667372703552
Step: 700  	Training loss: 0.3713502585887909
Step: 700  	Training accuracy: 0.77038

In [410]:
metric_out_df

,accuracy,precision,recall,f1_score,auc,loss,accuracy_val,loss_val,accuracy_test,loss_test,neurons,learning_rate,n_epochs,steps
0,0.778896,0.75,0.80198,0.804983,0.833151,0.35709,0.776734,0.509622,0.776438,0.487417,8.0,0.001,50000.0,1559.0


In [397]:
1+metric_out_df.loss_test/(np.log(0.5))

0    0.236025
Name: loss_test, dtype: float64

In [386]:
train_X.shape

(300, 1, 8)

In [387]:
def metrics_manual(file_path):
    

    # subj_num=12

    onehot_encoder = OneHotEncoder(sparse=False)
    # file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/vcheck/subject_num_"+str(subj_num)

    prob_train_df = pd.read_csv(file_path + "/prob_train_currentprevopts_neurons="+str(neurons)+".csv")
    prob_test_df = pd.read_csv(file_path + "/prob_test_currentprevopts_neurons="+str(neurons)+".csv")
    prob_val_df =  pd.read_csv(file_path + "/prob_val_currentprevopts_neurons="+str(neurons)+".csv")
    
    train_data_df = pd.read_csv(file_path+"/train_data.csv")
    test_data_df = pd.read_csv(file_path+"/test_data.csv")
    val_data_df = pd.read_csv(file_path+"/val_data.csv")


    prob_train =prob_train_df.values[:,1:]
    prob_test =prob_test_df.values[:,1:]
    prob_val =prob_val_df.values[:,1:]

    

    train_yy = train_data_df.Choice.values
    test_yy = test_data_df.Choice.values
    val_yy = val_data_df.Choice.values


    encode_categorical = train_yy.reshape(len(train_yy), 1)
    train_yy = onehot_encoder.fit_transform(encode_categorical)


    encode_categorical = test_yy.reshape(len(test_yy), 1)
    test_yy = onehot_encoder.fit_transform(encode_categorical)

    
    encode_categorical = val_yy.reshape(len(val_yy), 1)
    val_yy = onehot_encoder.fit_transform(encode_categorical)


    ############################################
    loss_train = -(np.dot(train_yy[:,0],np.log(prob_train[:,0])) + np.dot(train_yy[:,1],np.log(prob_train[:,1]) )) / train_yy.shape[0]
    acc_train = (np.dot(train_yy[:,0],prob_train[:,0]) + np.dot(train_yy[:,1],prob_train[:,1]))/train_yy.shape[0]
    pseudo_R2_train = 1 + loss_train/np.log(0.5)

    loss_test = -(np.dot(test_yy[:,0],np.log(prob_test[:,0])) + np.dot(test_yy[:,1],np.log(prob_test[:,1]) )) /test_yy.shape[0]
    acc_test = (np.dot(test_yy[:,0],prob_test[:,0]) + np.dot(test_yy[:,1],prob_test[:,1]))/150
    pseudo_R2_test  = 1 + loss_test/np.log(0.5)
    
    
    loss_val = -(np.dot(val_yy[:,0],np.log(prob_val[:,0])) + np.dot(val_yy[:,1],np.log(prob_val[:,1]) )) / val_yy.shape[0]
    acc_val = (np.dot(val_yy[:,0],prob_val[:,0]) + np.dot(val_yy[:,1],prob_val[:,1]))/val_yy.shape[0]
    pseudo_R2_val = 1 + loss_val/np.log(0.5)

    ############################################

    # pseudoR2_test  = 1 + (-((np.dot(test_yy[:,0],np.log(prob_test[:,0])) + np.dot(test_yy[:,1],np.log(prob_test[:,1]))))/150)/np.log(0.5)

    # print(acc_test)
    # print(pseudoR2_test)
    
    metric_manual_df= pd.DataFrame(np.array([subj_num,acc_train,loss_train,pseudo_R2_train, acc_val,loss_val,pseudo_R2_val,acc_test,loss_test,pseudo_R2_test,neurons]).reshape(-1,11),columns =["Subject_number","accuracy_train","loss_train","pseudoR2_train", "accuracy_val","loss_val","pseudoR2_val","accuracy_test","loss_test","pseudo_R2_test","neurons"])

    metric_manual_df.to_csv(file_path+"/metric_manual.csv")
    
    return metric_manual_df

In [419]:
for subj_num in range(28,42):
# for subj_num in range(28,42):
#     file_path ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/vcheck/subject_num_"+str(subj_num)
    file_path ="/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/v10chunks/subject_num_"+str(subj_num)


    metrics_manual_df = metrics_manual(file_path)

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

In [363]:
#     onehot_encoder = OneHotEncoder(sparse=False)
#     # file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/vcheck/subject_num_"+str(subj_num)

#     prob_train_df = pd.read_csv(file_path + "/prob_train_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_test_df = pd.read_csv(file_path + "/prob_test_currentprevopts_neurons="+str(neurons)+".csv")
#     prob_val_df =  pd.read_csv(file_path + "/prob_val_currentprevopts_neurons="+str(neurons)+".csv")
    
#     train_data_df = pd.read_csv(file_path+"/train_data.csv")
#     test_data_df = pd.read_csv(file_path+"/test_data.csv")
#     val_data_df = pd.read_csv(file_path+"/val_data.csv")


#     prob_train =prob_train_df.values[:,1:]
#     prob_test =prob_test_df.values[:,1:]
#     prob_val =prob_val_df.values[:,1:]

    

#     train_yy = train_data_df.Choice.values
#     test_yy = test_data_df.Choice.values
#     val_yy = val_data_df.Choice.values


#     encode_categorical = train_yy.reshape(len(train_yy), 1)
#     train_yy = onehot_encoder.fit_transform(encode_categorical)


#     encode_categorical = test_yy.reshape(len(test_yy), 1)
#     test_yy = onehot_encoder.fit_transform(encode_categorical)

    
#     encode_categorical = val_yy.reshape(len(val_yy), 1)
#     val_yy = onehot_encoder.fit_transform(encode_categorical)


#     ############################################
# #     loss_train = -(np.dot(train_yy[:,0],np.log(prob_train[:,0])) + np.dot(train_yy[:,1],np.log(prob_train[:,1]) )) / train_yy.shape[0]
# #     acc_train = (np.dot(train_yy[:,0],prob_train[:,0]) + np.dot(train_yy[:,1],prob_train[:,1]))/train_yy.shape[0]
# #     pseudo_R2_train = 1 + loss_train/np.log(0.5)

#     loss_test = -(np.dot(test_yy[:,0],np.log(prob_test[:,0])) + np.dot(test_yy[:,1],np.log(prob_test[:,1]) )) /test_yy.shape[0]
#     acc_test = (np.dot(test_yy[:,0],prob_test[:,0]) + np.dot(test_yy[:,1],prob_test[:,1]))/150
#     pseudo_R2_test  = 1 + loss_test/np.log(0.5)
    
    
#     loss_val = -(np.dot(val_yy[:,0],np.log(prob_val[:,0])) + np.dot(val_yy[:,1],np.log(prob_val[:,1]) )) / val_yy.shape[0]
#     acc_val = (np.dot(val_yy[:,0],prob_val[:,0]) + np.dot(val_yy[:,1],prob_val[:,1]))/val_yy.shape[0]
#     pseudo_R2_val = 1 + loss_val/np.log(0.5)

#     ###############

In [378]:
subj_num

41

In [416]:
metrics_manual_df = metrics_manual(file_path)

/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

In [375]:
# metric_out_df.accuracy_test

In [417]:
metrics_manual_df

,Subject_number,accuracy_train,loss_train,pseudoR2_train,accuracy_val,loss_val,pseudoR2_val,accuracy_test,loss_test,pseudo_R2_test,neurons
0,36.0,0.853764,0.214202,0.690972,0.770068,0.41171,0.406028,0.774565,0.38291,0.447577,8.0


In [253]:
file_path

'/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/ActualDataFitting/Pretraining/v6chunks/subject_num_41'

In [254]:
pd.read_csv(file_path+"/metric_manual.csv")

,Unnamed: 0,Subject_number,accuracy_train,loss_train,pseudoR2_train,accuracy_val,loss_val,pseudoR2_val,accuracy_test,loss_test,pseudo_R2_test,neurons
0,0,41.0,0.752035,0.357041,0.484899,0.702774,0.479203,0.308656,0.564067,0.537597,0.224412,8.0


In [283]:
metric_out_df

,accuracy,precision,recall,f1_score,auc,loss,accuracy_val,loss_val,accuracy_test,loss_test,neurons,learning_rate,n_epochs,steps
0,0.760037,0.775148,0.839744,0.790438,0.826735,0.367232,0.758883,0.476922,0.759083,0.529354,8.0,0.001,50000.0,3424.0


In [80]:
## Checking below

In [279]:
loss_train= - ((np.dot(train_y[:,0],np.log(prob_train[:,0])) + np.dot(train_y[:,1],np.log(prob_train[:,1]))))/300
print(loss_train)

0.44067895759247466


In [280]:
acc_train = ((np.dot(train_y[:,0],prob_train[:,0]) + np.dot(train_y[:,1],prob_train[:,1])))/300
acc_train

0.8935238987455766

In [284]:
loss_test= - ((np.dot(test_y[:,0],np.log(prob_test[:,0])) + np.dot(test_y[:,1],np.log(prob_test[:,1]))))/150
print(loss_test)

0.4236662762909496


In [285]:
acc_test = ((np.dot(test_y[:,0],prob_test[:,0]) + np.dot(test_y[:,1],prob_test[:,1])))/150
acc_test

0.5610535642039031

In [64]:
## rechecking below

In [85]:
file_path
train_data_df = pd.read_csv(file_path+"/train_data.csv")
test_data_df = pd.read_csv(file_path+"/test_data.csv")

In [86]:
# train_yy = train_data_df.Choice.values
# # print(train_yy.shape)

# encode_categorical = train_yy.reshape(len(train_yy), 1)

# train_yy = onehot_encoder.fit_transform(encode_categorical)
# ((np.dot(train_yy[:,0],prob_train_1[:,0]) + np.dot(train_yy[:,1],prob_train_1[:,1])))/300


# # train_data_df.Choice.values

In [87]:
## test

In [286]:

prob_test_df = pd.read_csv(file_path + "/prob_test_currentprevopts_neurons="+str(neurons)+".csv")
prob_test_1 =prob_test_df.values[:,1:]
test_yy = test_data_df.Choice.values


encode_categorical = test_yy.reshape(len(test_yy), 1)

test_yy = onehot_encoder.fit_transform(encode_categorical)
((np.dot(test_yy[:,0],prob_test[:,0]) + np.dot(test_yy[:,1],prob_test[:,1])))/150

# ((np.dot(test_yy[:,0],prob_test_1[:,0]) + np.dot(test_yy[:,1],prob_test_1[:,1])))/150


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


0.5610535642039031

In [69]:
## the above is correct. Below is wrong

In [45]:
train_X[0:10,:,-1]

array([[0.36571429],
       [0.82857143],
       [0.73142857],
       [1.        ],
       [0.6       ],
       [0.81714286],
       [0.85714286],
       [1.        ],
       [1.        ],
       [1.        ]])

In [8]:
### create a composite dataset comprising all subject's actual data

# task_mega_df = pd.DataFrame(); dopa_task_mega_df = pd.DataFrame();

for subj_num in range(28,41):
    print("Subject"+ str(subj_num))

    file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num)+"/"
    file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num) + "/experiment_data.csv"
    file_dopa_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/dopadata/subject_num_"+str(subj_num) +"/dopa_experiment_data.csv"
    
    task_df = pd.read_csv(file_name)
    dopa_task_df = pd.read_csv(file_dopa_name)

    task_df = add_releveant_features(task_df)
    dopa_task_df = add_releveant_features(dopa_task_df)
    
    
    task_mega_df = task_mega_df.append(task_df)
    dopa_task_mega_df = dopa_task_mega_df.append(dopa_task_df)

    

Subject28
Subject29
Subject30
Subject31
Subject32
Subject33
Subject34
Subject35
Subject36
Subject37
Subject38
Subject39
Subject40


In [9]:
task_net_df=pd.concat([task_mega_df,dopa_task_mega_df])
task_net_df.shape


(17458, 15)

In [18]:
task_df = task_net_df; dopa_task_df = task_net_df;
train_X, train_y, test_X, test_y,val_X,val_y = data_split(task_df,dopa_task_df)


(17458, 15)
12139
(12139, 8)
(12139, 1)
(2602, 8)
(2602, 1)
(2601, 8)
(2601, 1)


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

In [19]:
train_X

array([[[0.5       , 0.25714286, 0.82857143, ..., 0.        ,
         0.        , 0.36571429]],

       [[0.28571429, 0.        , 0.73142857, ..., 0.5       ,
         0.25714286, 0.82857143]],

       [[0.92857143, 0.61714286, 1.        , ..., 0.28571429,
         0.        , 0.73142857]],

       ...,

       [[1.        , 0.63428571, 1.        , ..., 0.92857143,
         0.28571429, 1.        ]],

       [[0.28571429, 0.        , 0.76      , ..., 1.        ,
         0.63428571, 1.        ]],

       [[0.5       , 0.37142857, 0.62857143, ..., 0.28571429,
         0.        , 0.76      ]]])

In [62]:
## create a composite dataset comprising all subject's actual data


# task_net_train_df = pd.DataFrame(); task_net_valid_df = pd.DataFrame();


# for subj_num in range(11,27):
for subj_num in range(28,42):

    print("Subject"+ str(subj_num))

    file_path = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num)+"/"
    file_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/placdata/subject_num_"+str(subj_num) + "/experiment_data.csv"
    file_dopa_name = "/Users/ritwik7/Dropbox (Personal)/Postdoc_UCL/DATA/rlab_incomplete_rewardSWB_code/by_RN/dopadata/subject_num_"+str(subj_num) +"/dopa_experiment_data.csv"
    
    task_df = pd.read_csv(file_name)
    dopa_task_df = pd.read_csv(file_dopa_name)

    task_df = add_releveant_features(task_df)
    dopa_task_df = add_releveant_features(dopa_task_df)
    
    
    
    task_net_train_df = task_net_train_df.append(pd.concat([task_df.loc[(task_df.TrialNum<=241) & (task_df.TrialNum>1)],dopa_task_df.loc[(dopa_task_df.TrialNum<=241) & (dopa_task_df.TrialNum>1)]]))
    task_net_valid_df = task_net_valid_df.append(pd.concat([task_df.loc[(task_df.TrialNum>241) ],dopa_task_df.loc[(dopa_task_df.TrialNum>241) ]]))

Subject28
Subject29
Subject30
Subject31
Subject32
Subject33
Subject34
Subject35
Subject36
Subject37
Subject38
Subject39
Subject40
Subject41


In [71]:
task_df = task_net_train_df
dopa_task_df  = task_net_valid_df

train_X, train_y, test_X, test_y,val_X,val_y = data_split(task_df,dopa_task_df)


(14400, 15)
(14400, 8)
(14400, 1)
(3540, 8)
(3540, 1)
(3540, 8)
(3540, 1)


/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)
/Users/ritwik7/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='au

In [73]:
task_net_train_df.shape[0]/30

480.0

In [65]:
task_net_valid_df.shape[0]/30

118.0

In [40]:
# task_mega_df

In [41]:
# dopa_task_mega_df